In [8]:
!pip install statsmodels

In [9]:
!pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from math import sqrt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [11]:
import random

random.seed(1680)

In [67]:
# Read the cab rides and weather data
cab_df = pd.read_csv("cab_rides.csv")
weather_df = pd.read_csv("weather.csv")

#print(cab_df.head(20))
#print(weather_df.head(20))

#print(weather_df.head())

# Convert time_stamp to datetime format
cab_df['date_time'] = pd.to_datetime(cab_df['time_stamp']/1000, unit='s')
weather_df['date_time'] = pd.to_datetime(weather_df['time_stamp'], unit='s')

#print(cab_df.head())

merged_df = pd.merge(cab_df, weather_df, on='date_time', how='right')

merged_df['price'].fillna(merged_df['price'].mean(), inplace=True)

merged_df['rain'].fillna(0, inplace=True)

#print(merged_df.isnull().sum())

merged_df.dropna(inplace=True)

#print("Shape after handling missing values:", merged_df.shape)

# Normalize price data by dividing price by distance
merged_df['normalized_price'] = merged_df['price'] / merged_df['distance']

merged_df['rush_hour'] = ((merged_df['date_time'].dt.hour >= 8) & (merged_df['date_time'].dt.hour < 10) |
                          ((merged_df['date_time'].dt.hour >= 16) & (merged_df['date_time'].dt.hour < 18))).astype(int)

merged_df['late_night'] = ((merged_df['date_time'].dt.hour >= 22) | (merged_df['date_time'].dt.hour < 6)).astype(int)

merged_df['day_of_week'] = merged_df['date_time'].dt.dayofweek

merged_df['heavy_rain'] = (merged_df['rain'] > 0.1).astype(int)

#print(merged_df[['rain','heavy_rain']].describe())

merged_df['weekend'] = ((merged_df['date_time'].dt.dayofweek == 4) & (merged_df['date_time'].dt.hour >= 18)) | \
                       (merged_df['date_time'].dt.dayofweek == 5) | \
                       ((merged_df['date_time'].dt.dayofweek == 6) & (merged_df['date_time'].dt.hour < 22)).astype(int)

merged_df['weekend'] = merged_df['weekend'].astype(float)

#print(merged_df.head(25))

X = merged_df[['rush_hour', 'late_night', 'weekend', 'heavy_rain']]
y = merged_df['surge_multiplier']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

#print(model.summary())
print(weather_df.shape[0])
print(merged_df.shape[0])

6276
55


In [2]:
conda install xgboost

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::conda==22.9.0=py39hca03da5_0
  - defaults/osx-arm64::sip==6.7.12=py39h313beb8_0
  - defaults/osx-arm64::anaconda-navigator==2.5.2=py39hca03da5_0
  - defaults/osx-arm64::clyent==1.2.2=py39hca03da5_1
  - defaults/osx-arm64::conda-repo-cli==1.0.75=py39hca03da5_0
  - defaults/osx-arm64::anaconda-client==1.12.2=py39hca03da5_0
  - defaults/osx-arm64::pyqt==5.15.10=py39h313beb8_0
  - defaults/osx-arm64::navigator-updater==0.4.0=py39hca03da5_1
  - defaults/noarch::conda-token==0.4.0=pyhd3eb1b0_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/scottpetersen/opt/miniconda3

  added / updated specs:
    - xgboost


The following NEW pa

In [45]:
import xgboost as xg
X = merged_df[['rush_hour', 'late_night', 'day_of_week']]
y = merged_df['surge_multiplier']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor()

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

ModuleNotFoundError: No module named 'xgboost'

In [9]:
cab_df = pd.read_csv("cab_rides.csv")
weather_df = pd.read_csv("weather.csv")

cab_df['date_time'] = pd.to_datetime(cab_df['time_stamp']/1000, unit='s')
weather_df['date_time'] = pd.to_datetime(weather_df['time_stamp'], unit='s')

merged_df = pd.merge(cab_df, weather_df, on='date_time', how='inner')

merged_df['rain'].fillna(0, inplace=True)

print(merged_df.head())

   distance cab_type   time_stamp_x              destination    source  price  \
0      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
1      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
2      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
3      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
4      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   

   surge_multiplier                                    id  \
0               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
1               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
2               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
3               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
4               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   

                             product_id    name           date_time   temp  \
0  6f72dfc5-27f1-42e8-84db-ccc7a75f6969  UberXL 2018-11-28 09:32:18 

In [15]:
merged_df['price'] = pd.to_numeric(merged_df['price'])

mean_price = merged_df['price'].mean()
std_price = merged_df['price'].std()
merged_df['normalized_price_z'] = (merged_df['price'] - mean_price) / std_price

merged_df['normalized_price_distance'] = merged_df['price'] / merged_df['distance']

mean_distance = merged_df['distance'].mean()
std_distance = merged_df['distance'].std()
merged_df['normalized_distance'] = (merged_df['distance'] - mean_distance) / std_distance

name_mapping = {name: idx for idx, name in enumerate(merged_df['name'].unique())}
merged_df['normalized_name'] = merged_df['name'].map(name_mapping)

print(merged_df.head())

   distance cab_type   time_stamp_x              destination    source  price  \
0      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
1      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
2      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
3      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   
4      1.22     Uber  1543397538000  Northeastern University  Back Bay   10.5   

   surge_multiplier                                    id  \
0               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
1               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
2               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
3               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   
4               1.0  85de4cd6-9dfb-42f1-a365-66ea58c2d3ca   

                             product_id    name  ... clouds  pressure rain  \
0  6f72dfc5-27f1-42e8-84db-ccc7a75f6969  UberXL  ...   0.42    991.7

In [20]:
numeric_columns = merged_df.select_dtypes(include=['float64', 'int64'])

name_column = merged_df[['name']]

numeric_and_name = pd.concat([numeric_columns, name_column], axis=1)

numeric_and_name = numeric_and_name.dropna()

print(numeric_and_name.dtypes)

distance                     float64
time_stamp_x                   int64
price                        float64
surge_multiplier             float64
temp                         float64
clouds                       float64
pressure                     float64
rain                         float64
time_stamp_y                   int64
humidity                     float64
wind                         float64
normalized_price_z           float64
normalized_price_distance    float64
normalized_distance          float64
normalized_name                int64
name                          object
dtype: object


In [22]:
X = numeric_columns[['date_time', 'temp', 'rain', 'humidity']]

X = sm.add_constant(X)

y = merged_df['price']

model = sm.OLS(y, X).fit()

print(model.summary())

KeyError: "['date_time'] not in index"